In [18]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.inspection import permutation_importance
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
import matplotlib.pyplot as plt
import joblib
import warnings
import sys

# 경고 무시 및 설정
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

# ==============================================================================
# 1. 데이터 로드 및 Alpha Feature 생성 (Feature Engineering)
# ==============================================================================
def load_and_process_data():
    print("🔄 1. 데이터 로딩 및 전처리 시작...")
    
    try:
        # 파일 로드
        df_price = pd.read_csv('eth_4hour_raw_price.csv')
        df_tech = pd.read_csv('eth_4hour_master.csv')
        df_daily = pd.read_csv('merge_data.csv')
        
        # 날짜 변환 및 인덱스 설정
        for df in [df_price, df_tech, df_daily]:
            df['date'] = pd.to_datetime(df['date'])
            df.sort_values('date', inplace=True)
            df.set_index('date', inplace=True)

        # 병합 (4시간봉 기준)
        df_merged = df_price.join(df_tech, how='inner', lsuffix='_price', rsuffix='_tech')
        df_daily_resampled = df_daily.reindex(df_merged.index, method='ffill') # 일봉을 4시간봉으로 확장
        data = df_merged.join(df_daily_resampled, how='left', rsuffix='_daily')
        
        # NaN 제거
        data = data.dropna()
        
        print("✨ 2. Alpha Feature(파생변수) 생성 중...")
        
        # [Alpha 1] 변동성 돌파 강도 (Trend * RSI)
        if 'ETH_TREND_SCORE' in data.columns and 'ETH_RSI_84' in data.columns:
            data['Alpha_Trend_Strength'] = data['ETH_TREND_SCORE'] * (data['ETH_RSI_84'] / 50)
            
        # [Alpha 2] 거래량 충격 (현재 거래량 / 20일 평균 거래량)
        if 'ETH_Volume' in data.columns:
            vol_ma = data['ETH_Volume'].rolling(window=120).mean() # 120봉 = 20일
            data['Alpha_Vol_Shock'] = data['ETH_Volume'] / (vol_ma + 1e-9)
            
        # [Alpha 3] 김프 변화량 (현재 김프 - 24시간 전 김프)
        if 'ETH_KP_Zscore' in data.columns:
            data['Alpha_KP_Delta'] = data['ETH_KP_Zscore'] - data['ETH_KP_Zscore'].shift(6)
            
        # [Alpha 4] 펀딩비 역주행 (음수 펀딩비인데 가격 상승)
        if 'fundingRate' in data.columns:
            price_change = data['ETH_Close'].pct_change()
            data['Alpha_Squeeze'] = (data['fundingRate'] < 0).astype(int) * price_change
            
        data = data.dropna()
        print(f"✅ 데이터 준비 완료. Shape: {data.shape}")
        return data

    except FileNotFoundError as e:
        print(f"❌ 파일 에러: {e}")
        sys.exit(1)

# ==============================================================================
# 2. 라벨링 및 1차 필터링 (Meta-Labeling Preparation)
# ==============================================================================
def apply_labeling_strategy(df):
    print("\n🏷️ 3. 라벨링 및 1차 신호 생성 (Triple Barrier)...")
    data = df.copy()
    
    # --- [설정] 전략 파라미터 (튜닝 가능) ---
    PT_RATIO = 3.5  # 익절 목표: ATR의 3.5배 (높은 수익 추구)
    SL_RATIO = 1.5  # 손절: ATR의 1.5배
    HORIZON = 24    # 보유 한계: 24봉 (4일)
    
    # 1차 필터 (Primary Signal): 너무 뻔한 자리는 거르고, 가능성 있는 자리만 1차 선별
    # 조건: 추세가 살아있고(Trend > 0.5) + 김프가 너무 안 높고(< 2.0) + 거래량이 좀 터짐
    c1 = data['ETH_TREND_SCORE'] > 0.5 if 'ETH_TREND_SCORE' in data.columns else True
    c2 = data['ETH_KP_Zscore'] < 2.0 if 'ETH_KP_Zscore' in data.columns else True
    c3 = data['Alpha_Vol_Shock'] > 0.8 if 'Alpha_Vol_Shock' in data.columns else True
    
    data['primary_signal'] = 0
    data.loc[c1 & c2 & c3, 'primary_signal'] = 1
    
    # ATR 계산 (없으면 생성)
    if 'ETH_ATR_84' in data.columns:
        atr = data['ETH_ATR_84']
    else:
        atr = data['ETH_High'] - data['ETH_Low']
    
    # Triple Barrier Labeling
    # (벡터화 연산 대신 가독성을 위해 루프 사용 - 속도 문제시 벡터화 필요)
    closes = data['ETH_Close'].values
    atr_vals = atr.values
    targets = np.zeros(len(data))
    
    # 신호가 있는 곳만 루프
    signal_indices = np.where(data['primary_signal'] == 1)[0]
    
    for i in signal_indices:
        if i + HORIZON >= len(data): continue
        
        entry = closes[i]
        upper = entry + (atr_vals[i] * PT_RATIO)
        lower = entry - (atr_vals[i] * SL_RATIO)
        
        label = 0 # 기본 0 (시간초과/손절)
        
        # Horizon 기간 동안의 고가/저가 확인
        # 단순화: 미래 Horizon 구간의 High가 Upper를 먼저 치냐, Low가 Lower를 먼저 치냐
        # 여기서는 간단히 "기간 내 최고가가 목표가 도달했으면 성공"으로 판별 (보수적 접근 필요시 수정)
        
        future_prices = closes[i+1 : i+1+HORIZON]
        
        # 익절 도달 여부
        hit_take_profit = np.any(future_prices >= upper)
        # 손절 도달 여부 (익절보다 먼저 쳤는지 확인해야 하나, 여기선 익절 도달하면 성공으로 간주)
        hit_stop_loss = np.any(future_prices <= lower)
        
        if hit_take_profit and not hit_stop_loss:
            label = 1
        elif hit_take_profit and hit_stop_loss:
             # 둘 다 닿으면 먼저 닿은거 체크해야함. 여기선 보수적으로 0
             # 정밀하게 하려면 step-by-step 비교 필요
             # 빠른 구현을 위해 Loop
             for p in future_prices:
                 if p <= lower: 
                     label = 0; break
                 if p >= upper: 
                     label = 1; break
        
        targets[i] = label

    data['target'] = targets
    
    print(f"   → 전체 샘플: {len(data)}")
    print(f"   → 1차 신호 수: {len(signal_indices)}")
    print(f"   → 수익 성공(Target=1): {int(targets.sum())} (승률: {targets.sum()/len(signal_indices)*100:.2f}%)")
    
    return data

# ==============================================================================
# 3. Feature Selection (여기가 핵심!)
# ==============================================================================
def select_best_features(X, y):
    print("\n🔬 4. [Advanced Feature Selection] 중요 변수 선별 중...")
    
    # 1단계: 상관관계(Correlation) 기반 제거 (유사한거 쳐내기)
    # 숫자형 컬럼만 선택
    X_num = X.select_dtypes(include=[np.number])
    
    # 너무 많은 컬럼은 계산이 오래 걸리므로, 분산이 0인거 제거
    X_num = X_num.loc[:, X_num.std() > 0]
    
    # 1. Spearman 상관계수 행렬
    corr = X_num.corr(method='spearman').abs()
    
    # 2. 계층적 군집화 (Hierarchical Clustering)
    # 거리가 가깝다 = 상관관계가 높다
    # NaN 처리 (상관관계 없는 경우)
    corr = corr.fillna(0)
    dist_matrix = 1 - corr
    dist_matrix = np.clip(dist_matrix, 0, 1) # 에러 방지
    
    # pdist 형식으로 변환 필요하지만, squareform으로 해결
    linkage = hierarchy.ward(squareform(dist_matrix))
    
    # 클러스터링 (거리 0.3 이하 = 상관계수 0.7 이상끼리 묶음)
    cluster_ids = hierarchy.fcluster(linkage, 0.3, criterion='distance')
    
    selected_feats = []
    cluster_map = {}
    for i, col in enumerate(X_num.columns):
        c_id = cluster_ids[i]
        if c_id not in cluster_map:
            cluster_map[c_id] = []
        cluster_map[c_id].append(col)
        
    # 각 클러스터에서 하나씩만 뽑기 (이름이 'Alpha' 들어간거 우선, 아니면 첫번째)
    for c_id, feats in cluster_map.items():
        alpha_feats = [f for f in feats if 'Alpha' in f]
        if alpha_feats:
            selected_feats.append(alpha_feats[0])
        else:
            selected_feats.append(feats[0])
            
    print(f"   → 1단계 완료: {X.shape[1]}개 -> {len(selected_feats)}개 (중복 제거)")
    
    # 2단계: MDA (Permutation Importance) - 모델 기반 검증
    # 가벼운 모델로 테스트
    X_reduced = X_num[selected_feats]
    
    model = lgb.LGBMClassifier(n_estimators=50, max_depth=3, random_state=42, verbose=-1)
    
    # 시계열 검증 (마지막 30% 데이터로 테스트)
    split_point = int(len(X_reduced) * 0.7)
    X_train, X_val = X_reduced.iloc[:split_point], X_reduced.iloc[split_point:]
    y_train, y_val = y.iloc[:split_point], y.iloc[split_point:]
    
    model.fit(X_train, y_train)
    
    # 순열 중요도 계산 (Scoring = Precision)
    # 주의: y_val에 1이 하나도 없으면 에러날 수 있음
    if y_val.sum() == 0:
        print("   ⚠️ 경고: 검증 데이터에 정답(1)이 없습니다. 랜덤 스플릿으로 대체합니다.")
        from sklearn.model_selection import train_test_split
        X_train, X_val, y_train, y_val = train_test_split(X_reduced, y, test_size=0.3, shuffle=True, stratify=y)
        model.fit(X_train, y_train)

    result = permutation_importance(
        model, X_val, y_val, n_repeats=5, random_state=42, n_jobs=-1, scoring='precision'
    )
    
    # 중요도가 0보다 큰 것만 선택
    imp_df = pd.DataFrame({'feature': X_reduced.columns, 'importance': result.importances_mean})
    final_features = imp_df[imp_df['importance'] > 0.001]['feature'].tolist() # 0.001 이상 기여한 것만
    
    if len(final_features) == 0:
        print("   ⚠️ 중요 변수가 없습니다. 상위 5개를 강제 선택합니다.")
        final_features = imp_df.sort_values('importance', ascending=False).head(5)['feature'].tolist()
        
    print(f"   → 2단계 완료: 최종 {len(final_features)}개 변수 선발 완료.")
    print(f"   ✅ Final Features: {final_features}")
    
    return final_features

# ==============================================================================
# 4. 최종 모델 학습 및 저장
# ==============================================================================
def train_final_model(df, features):
    print("\n🚀 5. 최종 모델 학습 (LightGBM)...")
    
    # 학습 데이터 준비
    mask = df['primary_signal'] == 1
    X = df.loc[mask, features]
    y = df.loc[mask, 'target']
    
    # 클래스 불균형 처리
    pos = y.sum()
    neg = len(y) - pos
    scale_pos_weight = neg / pos if pos > 0 else 1.0
    print(f"   → Class Weight 적용: {scale_pos_weight:.2f}")
    
    # 모델 설정 (일반화 성능 중시)
    model = lgb.LGBMClassifier(
        n_estimators=500,
        learning_rate=0.02,
        max_depth=4,
        num_leaves=20,
        scale_pos_weight=scale_pos_weight,
        colsample_bytree=0.7, # 변수 샘플링
        subsample=0.7,        # 데이터 샘플링
        random_state=42,
        verbose=-1
    )
    
    # 교차 검증 (성과 측정용)
    tscv = TimeSeriesSplit(n_splits=5)
    precision_scores = []
    
    print("\n   [Cross Validation Result]")
    for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        model.fit(X_train, y_train)
        
        # Threshold 튜닝 (보수적 접근: 0.6 이상일 때만 진입)
        prob = model.predict_proba(X_test)[:, 1]
        pred = (prob > 0.6).astype(int)
        
        prec = precision_score(y_test, pred, zero_division=0)
        count = pred.sum()
        real_win = y_test[pred==1].sum()
        
        print(f"   Fold {fold+1}: 진입 {count}회 -> 익절 {real_win}회 (승률: {prec:.2%})")
        if count > 0: precision_scores.append(prec)
            
    avg_prec = np.mean(precision_scores) if precision_scores else 0.0
    print(f"\n   🏆 평균 승률(Precision): {avg_prec:.2%}")
    
    # 전체 데이터 재학습 및 저장
    model.fit(X, y)
   # joblib.dump(model, 'eth_final_model.pkl')
    joblib.dump(features, 'eth_final_features.pkl') # 사용된 변수 리스트도 저장해야 함
    print("💾 모델 및 변수 리스트 저장 완료 ('eth_final_model.pkl', 'eth_final_features.pkl')")

# ==============================================================================
# Main 실행부
# ==============================================================================
if __name__ == "__main__":
    # 1. 데이터 로드 & Alpha Feature 추가
    df_raw = load_and_process_data()
    
    # 2. 라벨링 (수익/손실 정의)
    df_labeled = apply_labeling_strategy(df_raw)
    
    # 3. 변수 선택 (모델에 들어갈 X, y 준비)
    # 1차 신호가 뜬 데이터만 가지고 변수 선택을 해야 함 (실전 환경 모사)
    train_mask = df_labeled['primary_signal'] == 1
    X_candidates = df_labeled.loc[train_mask].drop(columns=['target', 'primary_signal', 'date'], errors='ignore')
    y_candidates = df_labeled.loc[train_mask, 'target']
    
    best_features = select_best_features(X_candidates, y_candidates)
    
    # 4. 학습 및 저장
    train_final_model(df_labeled, best_features)



🔄 1. 데이터 로딩 및 전처리 시작...
✨ 2. Alpha Feature(파생변수) 생성 중...
✅ 데이터 준비 완료. Shape: (14050, 719)

🏷️ 3. 라벨링 및 1차 신호 생성 (Triple Barrier)...
   → 전체 샘플: 14050
   → 1차 신호 수: 4279
   → 수익 성공(Target=1): 1185 (승률: 27.69%)

🔬 4. [Advanced Feature Selection] 중요 변수 선별 중...
   → 1단계 완료: 719개 -> 328개 (중복 제거)
   ⚠️ 중요 변수가 없습니다. 상위 5개를 강제 선택합니다.
   → 2단계 완료: 최종 5개 변수 선발 완료.
   ✅ Final Features: ['Alpha_Squeeze', 'BTC_Open', 'BTC_Volume', 'BTC_Bin_Volume', 'ETH_Open']

🚀 5. 최종 모델 학습 (LightGBM)...
   → Class Weight 적용: 2.61

   [Cross Validation Result]
   Fold 1: 진입 0회 -> 익절 0.0회 (승률: 0.00%)
   Fold 2: 진입 48회 -> 익절 18.0회 (승률: 37.50%)
   Fold 3: 진입 541회 -> 익절 88.0회 (승률: 16.27%)
   Fold 4: 진입 54회 -> 익절 20.0회 (승률: 37.04%)
   Fold 5: 진입 37회 -> 익절 22.0회 (승률: 59.46%)

   🏆 평균 승률(Precision): 37.57%
💾 모델 및 변수 리스트 저장 완료 ('eth_final_model.pkl', 'eth_final_features.pkl')
